In [1]:
from modelgym.models import CtBClassifier, LGBMClassifier
from modelgym.utils import XYCDataset
from modelgym.trainers import TpeTrainer
from modelgym.metrics import Accuracy
from modelgym.report import Report
from modelgym.utils import ModelSpace
from hyperopt import hp
import numpy as np
from sklearn.datasets import load_breast_cancer
from pathlib import Path
from wonderlandClient import ModelGymClient
from skopt.space import Integer, Categorical, Real
from modelgym.trainers import TpeTrainer, GPTrainer, RFTrainer, RandomTrainer
from modelgym.metrics import Accuracy, RocAuc, F1
import math


/home/igor/anaconda3/envs/yadro/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.4 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
client = ModelGymClient()
for param, val in client.config.items():
        print(param+":", val)

connect_to: 172.17.0.1:50051
local_project_root: ~/repo-storage/test
user: imusinov
azurefs_acc_name: mylake
azurefs_acc_key: nTYA+KhHEIuy2DVyG8uGuNev3qKGJ8Qm975hCkMgm+hGc7AW17RhnygFTKSNho5Iu8s3zwYcqxgrmte0tROBog==
azurefs_share: myshare
docker_image: musinov/training-image
client_cert: ~/.wonder/credentials/imusinov.crt
client_key: ~/.wonder/credentials/imusinov.key
ca_cert: ~/.wonder/credentials/wonderland.crt
max_msg_size_megabytes: 515


In [3]:
from wonderlandClient import generate_data
tmp_file_path = Path("~/repo-storage-test/test/DATA/temp_data_145earu.csv").expanduser()
generate_data(tmp_file_path)

In [4]:
data = load_breast_cancer()
y = np.array([data.target])
dataset = np.concatenate((data.data, y.T), axis=1)
n_features = len(data.data[0])
file = Path("~/repo-storage/test/DATA/temp_data_145earu.csv").expanduser()
np.savetxt(file, dataset,
           fmt='%.2f',
           header=','.join([str(x) for x in range(n_features)] + ['y']),
           delimiter=',')

In [5]:
catboost_space = [Integer(low=100, high=500, name='iterations'),
 Integer(low=1, high=11, name='depth'),
 Real(low=math.exp(-5), high=1e-1, prior='log-uniform', name='learning_rate'),
 Real(low=0, high=1, prior='uniform', transform='identity', name='rsm'),
 Categorical(categories=('Newton', 'Gradient'), prior=None, name='leaf_estimation_method'),
 Integer(low=1, high=10, name='l2_leaf_reg'),
 Real(low=0, high=2, prior='uniform', transform='identity', name='bagging_temperature')]

model_ctb = ModelSpace(CtBClassifier,
                   space=catboost_space,
                   space_update=False)

In [6]:
lgbm_space = [Real(low=math.exp(-7), high=1, prior='log-uniform', name='learning_rate'),
                 Integer(low=round(math.exp(1)), high=round(math.exp(7)), name='num_leaves'),
                 Real(low=0.5, high=1, name='feature_fraction'),
                 Real(low=0.5, high=1, name='bagging_fraction'),
                 Integer(low=1, high=round(math.exp(6)), name='min_data_in_leaf'),
                 Real(low=math.exp(-16), high=math.exp(5), prior='log-uniform', name='min_sum_hessian_in_leaf'),
                 Real(low=math.exp(-16), high=math.exp(2), prior='log-uniform', name='lambda_l1'),
                 Real(low=math.exp(-16), high=math.exp(2), prior='log-uniform', name='lambda_l2')]

model_lgbm = ModelSpace(LGBMClassifier,
                   space=lgbm_space,
                   space_update=False)

In [17]:
trainer = GPTrainer(model_lgbm)
best = trainer.crossval_optimize_params(RocAuc(), tmp_file_path, cv=3,
                                        opt_evals=2, metrics=[RocAuc()],
                                        verbose=False, workers=1, client=client)

In [15]:
print(trainer.get_best_results())

{'LGBMClassifier': {'result': {'output': {'loss': -0.7585252374286547, 'metric_cv_results': [{'roc_auc': 0.7544476327116213}, {'roc_auc': 0.7568191658247944}, {'roc_auc': 0.7643089137495489}], 'params': {'learning_rate': 0.10914282636359032, 'num_leaves': 369, 'feature_fraction': 0.5147056533501622, 'bagging_fraction': 0.7173875030980977, 'min_data_in_leaf': 163, 'min_sum_hessian_in_leaf': 0.7226686787838478, 'lambda_l1': 2.4813906521388783e-07, 'lambda_l2': 8.521391738506837e-05}, 'status': 'ok', 'loss_variance': 0.004202715484491106}, 'result_model_path': PosixPath('/home/igor/repo-storage/test/imusinov/model-ZGK868mei5sW/model.pickle')}, 'model_space': <modelgym.utils.model_space.ModelSpace object at 0x7f5aa0242390>}}


In [16]:
best_model = trainer.get_best_model()

Exception ignored in: <bound method Booster.__del__ of <lightgbm.basic.Booster object at 0x7f5aa0230208>>
Traceback (most recent call last):
  File "/home/igor/anaconda3/envs/yadro/lib/python3.6/site-packages/lightgbm/basic.py", line 1253, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


LightGBMError: b'Tree model string format error, should contain default_value field'